匿名流量识别器
----------------------
1. 使用一台机器开启热点服务器，然后使用Wireshark截获流量数据包。
2. 使用Scapy处理截获到的流量包，按双向通信的上行和下行流归纳数据。
3. 使用一个类，封装归纳总结的数据。
4. 使用Pandas的数据框，保存已经归纳得到的数据。
5. 使用随机森林算法，构建一个匿名流量分类器。

初步总结，由于初始数据集分布的不均匀，匿名流量的特征过于明显，使得分类器过拟合。

In [1]:
from scapy.all import *
filepath = "/home/john/桌面/Anonymous/mainflow.pcap"
packages = rdpcap(filepath)[IP]

In [9]:
datasets = dict()
hashes = set()
for i in range(len(packages)):
    p = packages[i]
    # print(p.time)
    if not hasattr(p, 'sport'):
        continue
    hashid = '' + str(p.proto) + str(p[IP].src) + str(p[IP].dst) + str(p.sport) + str(p.dport)
    hashid = hash(hashid)
    if not hashid in hashes:
        hashes.add(hashid)
        datasets[hashid] = [p]
        continue
    datasets[hashid].append(p)

# print(len(hashes))
print(len(datasets))

2108


In [11]:
import re

unique_ip = set()
for k,p in datasets.items():
    for i in range(len(p)):
        unique_ip.add(p[i][IP].src)
        
# print(len(unique_ip))
upflow_ip = set()
for ip in unique_ip:
    if re.match(r'172\.18\.12.108', ip):
        upflow_ip.add(ip)
        
for ip in upflow_ip:
    print(ip)

172.18.12.108


In [12]:
biflow = [] # 得到每个双向通信的流量包集合
showed = set()
for key in datasets.keys():
    temp = []
    showed.add(key)
    temp.append(datasets[key])
    p = datasets[key][0]
    re_key = '' + str(p.proto) + str(p[IP].dst) + str(p[IP].src) + str(p.dport) + str(p.sport)
    re_key = hash(re_key)
    if re_key in showed:
        continue
    if not re_key in hashes:
        biflow.append(temp)
    else:
        temp.append(datasets[re_key])
        biflow.append(temp)
        
print(len(biflow))

1853


In [13]:
biflow2 = [] # 分离上下行流量,元素1为上行流量，元素2为下行流量，去除只有一个方向的数据流量
for item in biflow:
    if len(item) > 1:
        src_ip_1 = item[0][0][IP].src
        src_ip_2 = item[1][0][IP].src
        if src_ip_1 in upflow_ip:
            # print(src_ip_1)
            biflow2.append(item)
        else:
            temp = []
            if src_ip_2 in upflow_ip:
                # print(src_ip_2)
                temp.append(item[1])
                temp.append(item[0])
                biflow2.append(temp)

print(len(biflow2))

255


In [14]:
for item in biflow2:
    src_ip = item[0][0][IP].src # 上行流的第1个数据包的ip地址
    if not src_ip in upflow_ip:
        assert False,"There is a ip not in upflow ips"
        
print(biflow2[1][0][0].show())

###[ Ethernet ]### 
  dst       = 00:19:06:56:1e:68
  src       = 74:27:ea:5a:61:0f
  type      = 0x800
###[ IP ]### 
     version   = 4
     ihl       = 5
     tos       = 0x0
     len       = 40
     id        = 44650
     flags     = DF
     frag      = 0
     ttl       = 64
     proto     = tcp
     chksum    = 0xdc45
     src       = 172.18.12.108
     dst       = 119.84.128.77
     \options   \
###[ TCP ]### 
        sport     = 36516
        dport     = https
        seq       = 3547925526
        ack       = 2669231483
        dataofs   = 5
        reserved  = 0
        flags     = A
        window    = 9476
        chksum    = 0x8e2c
        urgptr    = 0
        options   = []

None


In [15]:
class BiFlow:
    srcIP = '0.0.0.0'      # 源ip
    dstIP = '0.0.0.0'      # 目的ip
    proto = 0              # 协议号
    src_port = 0           # 源端口
    dst_port = 0           # 目的端口
    up_pkts = 0            # 上行数据包总数
    dw_pkts = 0            # 下行数据包总数
    up_pl_bytes = 0        # 上行载荷总量
    dw_pl_bytes = 0        # 下行载荷总量
    duration = 0           # 流持续时间
    up_avg_plsize = 0      # 上行载荷平均值
    dw_avg_plsize = 0      # 下行载荷平均值
    up_min_plsize = 0      # 上行最小载荷量
    dw_min_plsize = 0      # 下行最小载荷量
    up_max_plsize = 0      # 上行最大载荷量
    dw_max_plsize = 0      # 下行最大载荷量
    up_stdev_plsize = 0    # 上行载荷方差
    dw_stdev_plsize = 0    # 下行载荷方差
    up_avg_ipt = 0         # 上行数据包时间间隔平均值
    dw_avg_ipt = 0         # 下行数据包时间间隔平均值
    up_min_ipt = 0         # 上行最小时间间隔
    dw_min_ipt = 0         # 下行最小时间间隔
    up_max_ipt = 0         # 上行最大时间间隔
    dw_max_ipt = 0         # 下行最大时间间隔
    up_stdev_ipt = 0       # 上行时间间隔方差
    dw_stdev_ipt = 0       # 下行时间间隔方差
    
    ground_truth = 0       # 匿名流量分类，0:非匿名，1:匿名
    
    def __init__(self):
        pass

In [16]:
import time
import numpy as np

dataset = []
for item in biflow2:
    record = BiFlow()
    # print(len(item))
    record.srcIP = item[0][0][IP].src
    # print(record.srcIP)
    record.dstIP = item[0][0][IP].dst
    # print(record.dstIP)
    record.proto = item[0][0].proto
    # print(record.proto)
    record.src_port = item[0][0].sport
    # print(record.src_port)
    record.dst_port = item[0][0].dport
    # print(record.dst_port)
    record.up_pkts = len(item[0])
    # print(record.up_pkts)
    record.dw_pkts = len(item[1])
    # print(record.dw_pkts)
    
    # 处理上行流量
    times = [] # 先统计上行流量包时间, 再统计整体流量包的时间
    payloads = []    
    for pack in item[0]:
        record.up_pl_bytes += pack[IP].len
        times.append(pack.time)
        payloads.append(pack[IP].len)
    record.up_min_plsize = min(payloads)
    # print(record.up_min_plsize)
    record.up_max_plsize = max(payloads)
    # print(record.up_max_plsize)
    record.up_avg_plsize = np.mean(payloads)
    # print(record.up_avg_plsize)
    record.up_stdev_plsize = np.std(payloads)
    # print(record.up_stdev_plsize)
    # print(times)
    durtime = []
    for i in range(1,len(times)):
        durtime.append(times[i] - times[i-1])
    # print(durtime)
    if durtime:
        record.up_min_ipt = min(durtime)
        # print(record.up_min_ipt)
        record.up_max_ipt = max(durtime)
        # print(record.up_max_ipt)
        record.up_avg_ipt = np.mean(durtime)
        # print(record.up_avg_ipt)
        record.up_stdev_ipt = np.std(durtime)
        # print(record.up_stdev_ipt)
    
    # 处理下行流量
    dwflow_time = [] # 只统计下行流量包的时间
    payloads = []
    # print(len(item[1]))
    for pack in item[1]:
        record.dw_pl_bytes += pack[IP].len
        dwflow_time.append(pack.time)
        times.append(pack.time)
        payloads.append(pack[IP].len)
        # print(pack[IP].src)
    # print(payloads)
    record.dw_min_plsize = min(payloads)
    # print(record.dw_min_plsize)
    record.dw_max_plsize = max(payloads)
    # print(record.dw_max_plsize)
    record.dw_avg_plsize = np.mean(payloads)
    # print(record.dw_avg_plsize)
    record.dw_stdev_plsize = np.std(payloads)
    # print(record.dw_stdev_plsize)
    # print(dwflow_time)
    durtime = []
    for i in range(1,len(dwflow_time)):
        durtime.append(dwflow_time[i] - dwflow_time[i-1])
    # print(durtime)
    if durtime:
        record.dw_min_ipt = min(durtime)
        # print(record.dw_min_ipt)
        record.dw_max_ipt = max(durtime)
        # print(record.dw_max_ipt)
        record.dw_avg_ipt = np.mean(durtime)
        # print(record.dw_avg_ipt)
        record.dw_stdev_ipt = np.std(durtime)
        # print(record.dw_stdev_ipt)
    
    # 计算整体流的持续时间
    # print(times)
    record.duration = max(times) - min(times)
    # print(record.duration)
    
    # 将记录添加到数据集中
    dataset.append(record)
    
print(len(dataset))

255


In [17]:
for item in dataset:
    if item.dstIP == '120.79.247.42' and item.dst_port == 8989:
        item.ground_truth = 1

In [18]:
elm = dataset[0]
print(elm.srcIP)
print(elm.dstIP)
print(elm.proto)
print(elm.src_port)
print(elm.dst_port)
print(elm.up_pkts)
print(elm.dw_pkts)
print(elm.up_pl_bytes)
print(elm.dw_pl_bytes)
print(elm.duration)
print(elm.up_min_plsize)
print(elm.up_max_plsize)
print(elm.up_avg_plsize)
print(elm.up_stdev_plsize)
print(elm.dw_min_plsize)
print(elm.dw_max_plsize)
print(elm.dw_avg_plsize)
print(elm.dw_stdev_plsize)
print(elm.up_min_ipt)
print(elm.up_max_ipt)
print(elm.up_avg_ipt)
print(elm.up_stdev_ipt)
print(elm.dw_min_ipt)
print(elm.dw_max_ipt)
print(elm.dw_avg_ipt)
print(elm.dw_stdev_ipt)
print(elm.ground_truth)

172.18.12.108
114.114.114.114
17
44192
53
199
199
12679
31765
1394.7267589569092
56
86
63.71356783919598
6.364424272135666
94
350
159.62311557788945
42.72376703534181
1.3113021850585938e-05
100.00163793563843
7.0439243581559925
17.26564593615197
3.0994415283203125e-06
100.00149703025818
7.043927030129866
17.265636596510955
0


In [20]:
count = 0
for item in dataset:
    if item.ground_truth:
        print(item.srcIP)
        print(item.dstIP)
        print(item.src_port)
        print(item.dst_port)
        print(item.proto)
        print(' ')
        count += 1
        
print(count)

0


In [21]:
import pandas as pd
import numpy as np

feature_names = ['proto',           'up_pkts',         'dw_pkts',
                'up_pl_bytes',     'dw_pl_bytes',     'duration',
                'up_min_plsize',   'up_max_plsize',   'up_avg_plsize',
                'up_stdev_plsize', 'dw_min_plsize',   'dw_max_plsize',
                'dw_avg_plsize',   'dw_stdev_plsize', 'up_min_ipt',
                'up_max_ipt',      'up_avg_ipt',      'up_stdev_ipt',
                'dw_min_ipt',      'dw_max_ipt',      'dw_avg_ipt', 
                'dw_stdev_ipt',    'ground_truth']

data = []
for e in dataset:
    d = [e.proto,           e.up_pkts,         e.dw_pkts, 
         e.up_pl_bytes,     e.dw_pl_bytes,     e.duration, 
         e.up_min_plsize,   e.up_max_plsize,   e.up_avg_plsize, 
         e.up_stdev_plsize, e.dw_min_plsize,   e.dw_max_plsize,     
         e.dw_avg_plsize,   e.dw_stdev_plsize, e.up_min_ipt,
         e.up_max_ipt,      e.up_avg_ipt,      e.up_stdev_ipt,
         e.dw_min_ipt,      e.dw_max_ipt,      e.dw_avg_ipt,
         e.dw_stdev_ipt,    e.ground_truth]
    data.append(d)

data = np.array(data, dtype=np.float32).reshape(len(data), len(feature_names))
df = pd.DataFrame(data, columns=feature_names)
df['ground_truth'].replace(0.0, 'noanon', inplace = True)
df['ground_truth'].replace(1.0, 'anon', inplace = True)
df.head()

,proto,up_pkts,dw_pkts,up_pl_bytes,dw_pl_bytes,duration,up_min_plsize,up_max_plsize,up_avg_plsize,up_stdev_plsize,...,dw_stdev_plsize,up_min_ipt,up_max_ipt,up_avg_ipt,up_stdev_ipt,dw_min_ipt,dw_max_ipt,dw_avg_ipt,dw_stdev_ipt,ground_truth
0,17.0,199.0,199.0,12679.0,31765.0,1394.726807,56.0,86.0,63.713570,6.364424,...,42.723766,0.000013,100.001640,7.043924,17.265646,0.000003,100.001495,7.043927,17.265636,noanon
1,6.0,9.0,9.0,391.0,372.0,54.229977,40.0,71.0,43.444443,9.742360,...,3.771236,0.000019,10.240012,6.778747,4.553079,0.000020,10.240056,6.778511,4.553438,noanon
2,6.0,1.0,2.0,52.0,104.0,0.196212,52.0,52.0,52.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.196212,0.196212,0.196212,0.000000,noanon
3,6.0,4.0,3.0,208.0,156.0,82.766693,52.0,52.0,52.000000,0.000000,...,0.000000,0.049299,45.056000,27.588898,19.706043,37.661346,45.056046,41.358696,3.697351,noanon
4,6.0,4.0,3.0,208.0,156.0,82.762024,52.0,52.0,52.000000,0.000000,...,0.000000,0.044603,45.055977,27.587341,19.708231,37.661163,45.056122,41.358643,3.697479,noanon


In [22]:
df['is_train'] = np.random.uniform(0, 1, len(df)) <= .75
train, test = df[df['is_train']==True], df[df['is_train']==False]
features = df.columns[:22]
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_jobs=2)
y, _ = pd.factorize(train['ground_truth']) 
clf.fit(train[features], y)
target_names = np.array(['noanon', 'anon'])
preds = target_names[clf.predict(test[features])]
pd.crosstab(test['ground_truth'], preds, rownames=['actual'], colnames=['preds'])

preds,noanon
actual,
noanon,47
